In [ ]:
import sys
sys.path.append('..')

In [ ]:
from nhsmass.spectrum import Spectrum
from nhsmass.diff import Tmds
from nhsmass.chem import Reaction
from nhsmass.brutto import brutto_gen
import nhsmass.draw as draw

#### Discovery difference of soruce with product

In this example source was modifed by hydroquinone. So in product we expect to find source peak + quinone mass.

#### Load source

In [ ]:
sourse = Spectrum().read_csv('data/sourse.csv', mapper={'m/z':'mass','I':'intensity'}, sep=',', take_columns=['mass','intensity'])
gdf = brutto_gen(elems={'C':(1,40),'H':(0,80), 'O':(0,40),'N':(0,2)})
sourse = sourse.assign(generated_bruttos_table=gdf, rel_error=0.5)
draw.vk(sourse)

#### Load product

In [ ]:
product = Spectrum().read_csv('data/product.csv', mapper={'m/z':'mass','I':'intensity'}, sep=',', take_columns=['mass','intensity'])
gdf = brutto_gen(elems={'C':(1,40),'H':(0,80), 'O':(0,40),'N':(0,2)})
product = product.assign(generated_bruttos_table=gdf, rel_error=0.5)
draw.vk(product)

#### We can look difference spectrum

In [ ]:
draw.vk(product - sourse)

#### Now calculate total mass differnce statistic

We use restrict brutto gen because we expect to find six carbon atom difference. But you can study another atoms ranges

In [ ]:
tmds_spec = Tmds(spec=sourse).calc(other=product, p=0.2, C13_filter=False) 
tmds_spec = tmds_spec.assign(brutto_dict={'C':(6,7),'H':(0,13), 'O':(0,5)})
tmds_spec = tmds_spec.calc_mass()
draw.spectrum(tmds_spec)

In [ ]:
tmds_spec.table.sort_values(by='intensity', ascending=False)

Now we can see that most abudant mass-difference is C6H4O1-3, that contain in quinone fragment. We can propose that modification by quinone is occure.

#### Find source and product peaks that was modifed

Now we look which peaks in sourse was modifed by C6H4O2 fragment, and find them in product.

In [ ]:
gdf = brutto_gen(elems={'C':(6,7),'H':(4,5), 'O':(2,3)})

In [ ]:
gdf

In [ ]:
reaction = Reaction(sourse=sourse, product=product)

In [ ]:
reaction = reaction.find_modification(gdf)

In [ ]:
reaction.draw_modification()